In [1]:
import pandas as pd
import numpy as np

In [8]:
def no_dupes(df):
    
    #creating df of number of times a claim repeats

    repeats = df['ClaimID'].value_counts().to_frame().reset_index()

    #getting list of duplicate claims
    
    all_dupes = repeats[repeats['ClaimID']>1]
    dupes = all_dupes['index'].to_list()
   

    #getting one index value per duplicate claim

    goodi=[]
    for e in dupes:
        goodi.append(min(df[df['ClaimID'].isin([e])].index.to_list()))
    
    # full list of duplicate claim indicies

    alli=[]
    for e in dupes:
        alli = alli + df[df['ClaimID'].isin([e])].index.to_list()


    # take goodi from alli
    badi = [i for i in alli if i not in goodi]

    #drop duplicate indicies
    return(df.drop(index=badi).reset_index())

In [94]:
column_list = ['TotalFirstYearGrosskWh',
       'TotalFirstYearGrossTherm','TotalLifecycleGrosskWh',
       'TotalLifecycleGrossTherm','Budget']
def aggregate_me(df, by, lg, column_list):
    
    # making df usable

    df[by] = df[by].str.upper()
    
    #filtering for local government

    if lg: 

        lglist = ['Local Government (Counties)',
         'Local Government ()',
         'Local Government (Cities)',
         'Local Government ']

        df = df[df['Grouping'].isin(lglist)]


    # creating a total programs column

    prg = df.groupby([by, 'PrgID'], dropna = False).size().index.tolist() 
    #need to count the number of times a city/county shows up 

    e,f = [list(c) for c in zip(*prg)]

    ccp = pd.Series(e) # made a series so can use value_counts

    ccp = ccp.str.upper()

    tp = ccp.value_counts(dropna = False) #total programs per city
    
    # creating a total claims column
    
    df['total_claims'] = [1]*len(df)
    
    #aggregate!
    relevant_columns = column_list + ['total_claims']
    agg = df.groupby(by).sum()[relevant_columns]
    agg['total_programs'] = tp
    
    # Resource/OBF budget as percentage of total budget
    obf_budget = df.groupby([by, 'OBF_Flag']).Budget.sum().unstack().fillna(0).loc[:, True]
    resource_budget = df.groupby([by, 'Resource_Flag']).Budget.sum().unstack().fillna(0).loc[:, 1]
    agg['perc_obf_budget'] = (obf_budget/agg['Budget'])*100
    agg['perc_resource_budget'] = (resource_budget/agg['Budget'])*100
    
    return(agg.reset_index())

## Testing functions

In [4]:
cpuc = pd.read_csv('data/cpuc_with_budget_and_resource.csv')

C:\Users\audre\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5,7,12,17,19,22,23,24,27,47,48,56,57) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


get rid of Unnamed: 0 column because otherwise wayyyy too many duplicates for my no_dupes function to work without taking forever

In [5]:
copy = cpuc.copy()

copy.drop(columns='Unnamed: 0', inplace=True)

copy.drop_duplicates(inplace=True)

In [23]:
copy.shape

(599974, 72)

### no_dupes test

In [14]:
%%time

no_dupes(copy)

Wall time: 1.75 s


In [9]:
yay = no_dupes(copy)

In [27]:
yay['ClaimID'].value_counts()

SCE-2018-Q4-0113270                  1
SCG-2019-3711-12120676-3197087       1
SCE-2018-Q4-0101559                  1
PGE-2017-Q4-63376                    1
PGE-2018-Q4-75109                    1
                                    ..
SDGE-2019-3245-10977194-12234198S    1
SCG-2019-3805-12118934-3193552       1
SDGE-2019-3245-10981321-12259530     1
PGE-2018-Q4-88349                    1
SCE-2017-Q4-0199765                  1
Name: ClaimID, Length: 599968, dtype: int64

In [28]:
copy.shape

(599974, 72)

In [29]:
copy['ClaimID'].value_counts()

SCE-2018-Q2-0069119                 2
SCE-2018-Q4-0101513                 2
SCE-2017-Q4-0201431                 2
SCE-2018-Q4-0114147                 2
SCE-2017-Q4-0201694                 2
                                   ..
SCG-2019-3805-12118934-3193552      1
SDGE-2019-3245-10981321-12259530    1
PGE-2018-Q4-88349                   1
SDGE-2019-3224-11003707-12465587    1
SCE-2017-Q4-0199765                 1
Name: ClaimID, Length: 599968, dtype: int64

### aggregate_me test

In [131]:
aggregate_me(yay, 'SiteCity', True, ['Budget', 'TotalFirstYearGrosskWh'])

<ipython-input-130-54fe90ed6554>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Total Claims'] = [1]*len(df)


,Budget,TotalFirstYearGrosskWh,Total Claims,Total Programs
SiteCity,,,,
ACAMPO,17684.568509,48664.726740,6,1
ADELANTO,27320.063231,13576.844232,4,1
ALAMO,48785.542683,73049.392600,15,1
ALBANY,60926.193528,153765.922000,35,2
ALHAMBRA,48465.447601,21630.949392,7,1
...,...,...,...,...
WOODLAND,278715.826268,526910.057920,358,2
YORBA LINDA,230594.633505,100589.028000,12,1
YOSEMITE NATIONAL PARK,18234.382541,8894.000000,4,1


In [132]:
yay[yay['SiteCity']=='ACAMPO'].PrgID.value_counts()

PGE21012     9
PGE211030    6
Name: PrgID, dtype: int64

In [45]:
yay.columns

Index(['ClaimID', 'PrgID', 'Sector_x', 'SiteCity', 'SiteZipCode', 'SiteID',
       'NAICSCode', 'BldgHVAC', 'BldgLoc', 'BldgType', 'BldgVint',
       'E3ClimateZone', 'E3GasSavProfile', 'E3GasSector',
       'E3MeaElecEndUseShape', 'E3TargetSector', 'ImplementationID',
       'InstallationDate', 'NAICSBldgType', 'NumUnits', 'OBF_Flag',
       'PrgElement', 'RateScheduleElec', 'RateScheduleGas', 'REN_Flag',
       'Residential_Flag', 'SchoolIdentifier', 'TotalFirstYearGrosskW',
       'TotalFirstYearGrosskWh', 'TotalFirstYearGrossTherm',
       'TotalGrossIncentive', 'TotalGrossMeasureCost',
       'TotalGrossMeasureCost_ER', 'TotalLifecycleGrosskW',
       'TotalLifecycleGrosskWh', 'TotalLifecycleGrossTherm', 'Upstream_Flag_x',
       'WaterOnly_Flag', 'PrgYear', 'Claim ID', 'Program ID',
       'First Year Gross kWh', 'First Year Gross Therm', 'Budget', 'PA',
       'ProgramName', 'Grouping', 'County', 'Selected', 'PrimarySector',
       'Sector_y', 'BP Sector', 'ProgramImplementer', 

In [64]:
columns = ['Budget', 'TotalFirstYearGrosskWh', 'TotalFirstYearGrossTherm','TotalLifecycleGrosskWh', 'TotalLifecycleGrossTherm', 'OBF_Flag', 'Resource_Flag', 'Total Claims']
test = aggregate_me(yay, 'SiteCity', False, columns)
test

,SiteCity,Budget,TotalFirstYearGrosskWh,TotalFirstYearGrossTherm,TotalLifecycleGrosskWh,TotalLifecycleGrossTherm,OBF_Flag,Resource_Flag,Total Claims,Total Claims,Total Programs
0,ACAMPO,4.031116e+04,5.135173e+04,4187.109670,6.162207e+05,84933.316040,0,15,15.0,15.0,2
1,ACTON,1.157423e+05,1.168743e+05,724.745606,8.603851e+05,10175.558937,0,62,52.0,52.0,4
2,ADELANTO,8.736705e+05,2.389739e+06,425.198725,1.462304e+07,5461.256061,0,568,567.0,567.0,7
3,AGOURA,7.469575e+03,2.984935e+04,-114.149175,1.502801e+05,-574.860718,0,8,8.0,8.0,1
4,AGOURA HILLS,2.134583e+05,3.737607e+05,23780.921320,1.908289e+06,305775.918080,0,115,101.0,101.0,7
...,...,...,...,...,...,...,...,...,...,...,...
1021,YOSEMITE NATIONAL PARK,2.547790e+04,1.390673e+04,-36.737900,1.534581e+05,-234.496500,0,11,11.0,11.0,2
1022,YOUNTVILLE,2.665221e+04,5.969097e+04,4415.974000,5.697245e+05,80364.513640,0,80,80.0,80.0,3
1023,YUBA CITY,1.378842e+06,1.997491e+06,23805.462403,2.223440e+07,294117.615441,52,1046,1046.0,1046.0,8
1024,YUCAIPA,4.099928e+05,5.131031e+05,37881.590152,4.685987e+06,421490.942175,0,192,117.0,117.0,12


# **THEY WORK!!!!**
jk don't get too excited there are still many an issue

want to perfect the aggregate one, ie get obf_claims and resource_claims data as percentage of budget and total programs

will have parameter obf=True that will determine whether or not to go through with obf/resource claims agg, see obf_resource_budget.ipynb

In [58]:
test[test['Resource_Flag']>test['Total Claims']]

,SiteCity,Budget,TotalFirstYearGrosskWh,TotalFirstYearGrossTherm,TotalLifecycleGrosskWh,TotalLifecycleGrossTherm,OBF_Flag,Resource_Flag,Total Claims,Total Programs
1,ACTON,115742.339742,1.168743e+05,724.745606,8.603851e+05,1.017556e+04,0,62,52.0,4
2,ADELANTO,873670.472213,2.389739e+06,425.198725,1.462304e+07,5.461256e+03,0,568,567.0,7
4,AGOURA HILLS,213458.272103,3.737607e+05,23780.921320,1.908289e+06,3.057759e+05,0,115,101.0,7
10,ALHAMBRA,857252.068024,1.267773e+06,36953.769382,1.057979e+07,5.411042e+05,0,459,433.0,10
11,ALISO VIEJO,460812.732166,1.588288e+06,25418.428892,1.211626e+07,3.725117e+05,0,381,220.0,11
...,...,...,...,...,...,...,...,...,...,...
1013,WOODLAND HILLS,13650.179453,3.245400e+04,3486.899995,5.354910e+05,4.273785e+04,0,8,4.0,2
1014,WOODLAND HLS,374392.275016,-7.173176e+02,97672.390686,-1.788276e+04,1.235553e+06,0,105,34.0,5
1020,YORBA LINDA,831678.831350,6.762897e+05,38133.843571,4.607387e+06,4.400422e+05,12,780,691.0,11
1024,YUCAIPA,409992.830365,5.131031e+05,37881.590152,4.685987e+06,4.214909e+05,0,192,117.0,12


In [129]:
aggregate_me(copy, 'SiteCity', True, ['Budget', 'Resource_Flag'])

<ipython-input-128-c6422af5c10a>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Total Claims'] = [1]*len(df)


,SiteCity,Budget,Resource_Flag,Total Claims,Total Programs
0,ACAMPO,17684.568509,6,6,1
1,ADELANTO,27320.063231,4,4,1
2,ALAMO,48785.542683,15,15,1
3,ALBANY,60926.193528,35,35,2
4,ALHAMBRA,48465.447601,7,7,1
...,...,...,...,...,...
532,WOODLAND,278715.826268,358,358,2
533,YORBA LINDA,230594.633505,12,12,1
534,YOSEMITE NATIONAL PARK,18234.382541,4,4,1
535,YOUNTVILLE,4901.835174,57,57,2


## something funky is going on with the total claims column, and it is not the fault of no_dupes

In [112]:
def aggregate_me_2(df, by, lg, column_list):
    
    # making df usable

    df[by] = df[by].str.upper()
    
    # creating a total claims column

    l = [1]*len(df)
    total_claims = pd.Series(l).rename('Total Claims from function')
    df = df.join(total_claims)
    return(df)
    #filtering for local government

    if lg: 

        lglist = ['Local Government (Counties)',
         'Local Government ()',
         'Local Government (Cities)',
         'Local Government ']

        df = df[df['Grouping'].isin(lglist)]


    # creating a total programs column

    prg = df.groupby([by, 'PrgID'], dropna = False).size().index.tolist() 
    #need to count the number of times a city/county shows up 

    e,f = [list(c) for c in zip(*prg)]

    ccp = pd.Series(e) # made a series so can use value_counts

    ccp = ccp.str.upper()

    tp = ccp.value_counts(dropna = False) #total programs per city
    
#     # creating a total claims column

#     l = [1]*len(df)
#     total_claims = pd.Series(l).rename('Total Claims from function')
#     df = df.join(total_claims)
#     return(df)
   
    #aggregate!
    relevant_columns = column_list + ['Total Claims from function']
    return(df.groupby(by).sum()[relevant_columns]
           .join(tp.rename('Total Programs'))
           .reset_index())

In [74]:
test2 = aggregate_me_2(copy, 'SiteCity', True, ['Budget'])

In [97]:
test2['Total Claims from function'].value_counts(dropna=False)

1.0    123013
NaN     69080
Name: Total Claims from function, dtype: int64

In [98]:
len(test2)

192093

In [100]:
lglist = ['Local Government (Counties)',
         'Local Government ()',
         'Local Government (Cities)',
         'Local Government ']

len(copy[copy['Grouping'].isin(lglist)])


192093

In [102]:
test2.fillna(0)[test2.fillna(0)['Total Claims from function'] == 0 ]

,ClaimID,PrgID,Sector_x,SiteCity,SiteZipCode,SiteID,NAICSCode,BldgHVAC,BldgLoc,BldgType,...,Upstream_Flag_y,Midstream_Flag,Downstream_Flag,DirectInstall,Audit_Flag,Financing,ParentProgram,Exclude_From_Budget,Exclude_From_CE,Total Claims from function
192217,PGE-2018-Q1-17442,PGE211007,Com,EMERYVILLE,94608,PGE-1159990,922600,Any,3A,COM,...,False,False,True,Partial,True,None,Government Partnership Programs,False,False,0.0
192218,PGE-2018-Q1-18276,PGE211009,Com,FREMONT,94538,PGE-1194446,921190,Any,3B,COM,...,False,False,True,Partial,True,None,Government Partnership Programs,False,False,0.0
192219,PGE-2018-Q1-17496,PGE211007,Com,EMERYVILLE,94608,PGE-1159990,922600,Any,3A,COM,...,False,False,True,Partial,True,None,Government Partnership Programs,False,False,0.0
192222,PGE-2018-Q1-17494,PGE211009,Com,FREMONT,94538,PGE-1148165,921190,Any,3B,COM,...,False,False,True,Partial,True,None,Government Partnership Programs,False,False,0.0
192226,PGE-2018-Q1-18453,PGE211009,Com,FREMONT,94538,PGE-1148165,921190,Any,3B,COM,...,False,False,True,Partial,True,None,Government Partnership Programs,False,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746308,SDGE-2019-3277-2019Q4PROGRAMCOSTPLACEHOLDER,SDGE3277,Public,SAN DIEGO,92123,SDGE-2019-3277-2019Q4,0,Any,CZ07,Res,...,False,False,False,No,True,Other Financing,Local Partnerships,False,False,0.0
746309,SDGE-2019-3278-2019Q1ProgramCostPlaceholder,SDGE3278,Public,SAN DIEGO,92123,SDGE-2019-3278-2019Q1,0,Any,CZ07,Res,...,False,False,False,No,True,Other Financing,Local Partnerships,False,False,0.0
746310,SDGE-2019-3278-2019Q2ProgramCostPlaceholder,SDGE3278,Public,SAN DIEGO,92123,SDGE-2019-3278-2019Q2,0,Any,CZ07,Res,...,False,False,False,No,True,Other Financing,Local Partnerships,False,False,0.0
746311,SDGE-2019-3278-2019Q3ProgramCostPlaceholder,SDGE3278,Public,SAN DIEGO,92123,SDGE-2019-3278-2019Q3,0,Any,CZ07,Res,...,False,False,False,No,True,Other Financing,Local Partnerships,False,False,0.0


In [110]:
test3 = aggregate_me_2(copy, 'SiteCity', True, ['Budget', 'Resource_Flag'])

In [111]:
test3[test3['Resource_Flag']>test3['Total Claims from function']]

,SiteCity,Budget,Resource_Flag,Total Claims from function,Total Programs


In [113]:
aggregate_me_2(copy, 'SiteCity', False, ['Budget', 'Resource_Flag']).fillna(0)['Total Claims from function'].value_counts()

1.0    453575
0.0    146399
Name: Total Claims from function, dtype: int64

In [118]:
l = [1]*len(copy)
total_claims = pd.Series(l).rename('Total Claims')
test4 = copy.join(total_claims)

In [114]:
copy.shape

(599974, 72)

In [120]:
len(copy)

599974

In [116]:
nodups = copy.copy()

In [119]:
test4['Total Claims']

0         1.0
1         1.0
2         1.0
3         1.0
4         1.0
         ... 
746368    NaN
746369    NaN
746370    NaN
746371    NaN
746372    NaN
Name: Total Claims, Length: 599974, dtype: float64

so it looks like those nan values are left over from the og version of cpuc, the one with all of the duplicates, and for some reason those duplicate rows haven't gone away?

# ok now that i've got the total claims column figured out gotta figure out total programs column cuz its also incorrect

actually I think it is fine because the thing i was testing on (yay) doesn't have local government programs filtered out but the ag does

In [10]:
ag = aggregate_me(yay, 'SiteCity', True, ['Budget'])

<ipython-input-3-54fe90ed6554>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Total Claims'] = [1]*len(df)


In [33]:
len(yay[yay['SiteCity']=='ACTON']['PrgID'].value_counts())

4

In [137]:
ag.index

Index(['ACAMPO', 'ADELANTO', 'ALAMO', 'ALBANY', 'ALHAMBRA', 'ALTA', 'ALTADENA',
       'AMERICAN CANYON', 'ANAHEIM', 'ANDERSON',
       ...
       'WILLOW CREEK', 'WILLOWS', 'WINDSOR', 'WINTON', 'WOODACRE', 'WOODLAND',
       'YORBA LINDA', 'YOSEMITE NATIONAL PARK', 'YOUNTVILLE', 'YUBA CITY'],
      dtype='object', name='SiteCity', length=537)

^ takes way too long cuz that for loop

In [23]:
yay.groupby(['SiteCity', 'PrgID']).sum().index.tolist().to_Series()

AttributeError: 'list' object has no attribute 'to_Series'

In [30]:
ag.iloc[372, :]

Budget            7081.374945
Total Claims        13.000000
Total Programs       1.000000
Name: PLACENTIA, dtype: float64

In [31]:
agg = aggregate_me(yay, 'SiteCity', False, ['Budget'])

In [32]:
agg.head()

,Budget,Total Claims,Total Programs
SiteCity,,,
ACAMPO,40311.155034,15,2
ACTON,115742.339742,62,4
ADELANTO,873670.472213,568,7
AGOURA,7469.575278,8,1
AGOURA HILLS,213458.272103,115,7
